In [1]:
import librosa
import pyworld as pw
import scipy
import numpy as np
from PIL import Image
from pathlib import Path
from joblib import Parallel,delayed
from sklearn.preprocessing import normalize
import pickle
from IPython import embed
from tqdm import tqdm_notebook as tqdm

# import argparse
# import fastai
# from fastai.vision import *
# from fastai.callbacks import *
# from fastai.utils.mem import *
# from torchvision.models import resnet18

import util


In [2]:
path=Path("D:/bulk/dataset/audio/Anonymous-VC")
path_raw =                  path/"10_raw_yuduki-yukari"
path_prep_original =        path/"21_prep_original"
path_prep_original_10sec =  path/"22_prep_original_10sec"
path_prep_anonymous =       path/"31_prep_anonymous"
path_prep_anonymous_10sec = path/"32_prep_anonymous_10sec"
print(path)


D:\bulk\dataset\audio\Anonymous-VC


In [3]:
# reference : https://qiita.com/Yuhsak/items/1e8533343cf5458e2e08#joblib%E3%81%A7%E4%B8%A6%E5%88%97%E5%8C%96%E3%81%99%E3%82%8B
def preprocess_input(p:Path):
    src ,fs= librosa.load(p.resolve(), sr=util.conf.sample_ratio)
    if  util.conf.prep_audio_dataset_second is not None:
        src = util.clip_audio_length(src,fs,second=util.conf.prep_audio_dataset_second)

    # preprocess src audio
    src_abs, src_phase = util.convert_to_spectrogram(src)
    src_abs ,src_phase = util.padding_spectrogram(src_abs), util.padding_spectrogram(src_phase)
    # src_abs,_scale_factor,_offset= util.normalize(src_abs)
    
    # preprocess anonymized audio
    anonymized = util.anonymization(fs, src)
    anonymized_abs, anonymized_phase = util.convert_to_spectrogram(anonymized)
    anonymized_abs, anonymized_phase = util.padding_spectrogram(anonymized_abs), util.padding_spectrogram(anonymized_phase)
    # anonymized_abs,_scale_factor,_offset= util.normalize(anonymized_abs)
    
    with open(path_prep_original/(p.name+".pkl"),'wb') as f:
        pickle.dump(np.array([src_abs,src_phase]),f)
    with open(path_prep_anonymous/(p.name+".pkl"),'wb') as f:
        pickle.dump(np.array([anonymized_abs,anonymized_phase]),f)

    print("processed {}".format(p.name+".pkl"))
    return src_abs, src_phase, anonymized_abs, anonymized_phase


In [4]:
# try to preprocess 1 audio file.
p = list(path_raw.glob("**/*.wav"))[0]
src_abs, src_phase, anonymized_abs, anonymized_phase = preprocess_input(p)

nsa,*_ = util.normalize(src_abs)
nsp,*_ = util.normalize(src_phase)
util.save_spectrogram_asImage(nsa,nsp,path_prep_original/(p.name+"_image.png"))
naa,*_ = util.normalize(anonymized_abs)
nap,*_ = util.normalize(anonymized_phase)
util.save_spectrogram_asImage(naa,nap,path_prep_anonymous/(p.name+"_image.png"))


processed ATR503_001.wav.pkl


In [6]:
Parallel(n_jobs=-1)( [delayed(preprocess_input)(p) for p in tqdm(path_raw.glob("**/*.wav"))] )

[(array([[-7.9645580e-01, -1.0588450e+00, -2.3058422e+00, ...,
           4.8147635e-06,  4.8147635e-06,  4.8147635e-06],
         [-8.6163938e-01, -1.1296463e+00, -1.6453348e+00, ...,
           4.8147635e-06,  4.8147635e-06,  4.8147635e-06],
         [-9.3360811e-01, -1.3064331e+00, -1.6466478e+00, ...,
           4.8147635e-06,  4.8147635e-06,  4.8147635e-06],
         ...,
         [-2.5685246e+00, -2.8386538e+00, -3.6979713e+00, ...,
           4.8147635e-06,  4.8147635e-06,  4.8147635e-06],
         [-2.5649991e+00, -2.8882236e+00, -3.9659579e+00, ...,
           4.8147635e-06,  4.8147635e-06,  4.8147635e-06],
         [-2.5616596e+00, -2.8477950e+00, -4.2121239e+00, ...,
           4.8147635e-06,  4.8147635e-06,  4.8147635e-06]], dtype=float32),
  array([[ 0.0000000e+00,  0.0000000e+00,  3.1415927e+00, ...,
           0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
         [ 3.1415927e+00, -1.8789968e+00, -1.0915173e+00, ...,
           0.0000000e+00,  0.0000000e+00,  0.0000000